In [1]:
import os
from tqdm import tqdm
import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
from joblib import Parallel, delayed

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)


In [3]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
0,578,2013-01-02,1,103665,2.0,Unk,BREAD/BAKERY,2712,1,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_103665
1,579,2013-01-02,1,105574,8.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105574
2,580,2013-01-02,1,105575,15.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105575
3,581,2013-01-02,1,105577,2.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105577
4,582,2013-01-02,1,105737,2.0,Unk,GROCERY I,1044,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105737


In [5]:
df.shape

(2562153, 27)

In [6]:
len(set(df.item_nbr))

3567

In [7]:
np.min(df.date)

'2013-01-02'

In [8]:
np.max(df.date)

'2017-08-15'

In [9]:
np.datetime64(np.max(df.date)) - np.datetime64(np.min(df.date))

numpy.timedelta64(1686,'D')

In [10]:
365 * 4

1460

In [11]:
def set_recent_stats(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_mean'.format(days)] = mean
    df2.loc[0,'us_{}_med'.format(days)] = median
    df2.loc[0,'us_{}_std'.format(days)] = std
    df2.loc[0,'us_{}_skew'.format(days)] = skew
    df2.loc[0,'us_{}_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo2(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo3(df2, date, days, fn, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety

def set_recent_stats_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_h_mean'.format(days)] = mean
    df2.loc[0,'us_{}_h_med'.format(days)] = median
    df2.loc[0,'us_{}_h_std'.format(days)] = std
    df2.loc[0,'us_{}_h_skew'.format(days)] = skew
    df2.loc[0,'us_{}_h_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_h_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_per(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_per_mean'.format(days)] = mean
    df2.loc[0,'us_{}_per_med'.format(days)] = median
    df2.loc[0,'us_{}_per_std'.format(days)] = std
    df2.loc[0,'us_{}_per_skew'.format(days)] = skew
    df2.loc[0,'us_{}_per_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_per_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_nper(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_nper_mean'.format(days)] = mean
    df2.loc[0,'us_{}_nper_med'.format(days)] = median
    df2.loc[0,'us_{}_nper_std'.format(days)] = std
    df2.loc[0,'us_{}_nper_skew'.format(days)] = skew
    df2.loc[0,'us_{}_nper_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_nper_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow(df2, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_tbn'.format(days, dow)] = ten_by_ninety

def set_recent_stats_promo_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_ph_mean'.format(days)] = mean
    df2.loc[0, 'us_{}_ph_med'.format(days)] = median
    df2.loc[0, 'us_{}_ph_std'.format(days)] = std
    df2.loc[0, 'us_{}_ph_skew'.format(days)] = skew
    df2.loc[0, 'us_{}_ph_kurt'.format(days)] = kurtosis
    df2.loc[0, 'us_{}_ph_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow_promo_hol(df1, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_ph_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_ph_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_ph_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_ph_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_ph_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_ph_tbn'.format(days, dow)] = ten_by_ninety


In [12]:
# df_h = pd.read_csv('../data/holidays_events.csv')

In [13]:
# df_h.type.value_counts()

In [14]:
# df_h1 = df_h[(df.date.apply(np.datetime64)) > np.datetime64('2017-08-15')]

In [15]:
# df.onpromotion.value_counts()

In [16]:
# df.onpromotion[df.onpromotion==True].value_counts()

In [17]:
# df.h_type.value_counts()

In [18]:
# df.h_type[df.h_type=='Work Day'].value_counts()

In [19]:
# df.h_type[df.h_type != 'Work Day'].value_counts()

In [20]:
#df_h.h_type[df_h.type == 'Additional'] = 'Holiday'

In [21]:
recent_days = [30, 90, 180, 365, 730]

In [22]:
def calculate_recent_stats(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        

def calculate_recent_stats_promo(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo3(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
        
def calculate_recent_stats_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.h_type != 'Work Day')]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
 
def calculate_recent_stats_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 1)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_per(df3, date,days, mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_not_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_nper(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
#         data = df.unit_sales[(df.dts < np.datetime64(date)) & 
#                              (df.dts >= past_date) & (df.perishable == 0)]
        
        data = df.unit_sales[(df.date.astype('str') < str(date)) & 
                             (df.date.astype('str') >= str(past_date)) & (df.perishable == 0) ]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_nper(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_nper(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_dow(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_promo_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_dow_promo_hol(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 1
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 2):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & 
                             (df.onpromotion == True) & 
                             (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,dow,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow_promo_hol(df3, date,days,dow,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3

In [23]:
print(df.unit_sales.mean())

5.52075222249


In [24]:
past_date = (np.datetime64('2016-01-01') - np.timedelta64(2, 'D'))

In [25]:
# df.unit_sales[(df.date.apply(np.datetime64) >= past_date) & 
#               (df.date.apply(np.datetime64) < np.datetime64('2016-01-01'))].mean()

In [26]:
df1 = df.set_index('date')

In [27]:
s1 = df1.loc['2015-12-31', 'unit_sales']
s2 = df1.loc['2015-12-30', 'unit_sales']

In [28]:
s = pd.concat([s1, s2], axis=0)

In [29]:
s.mean()

6.5914254756871031

In [30]:
gc.enable()

In [31]:
# import os
# x = {i for i in range(20)}
# os.sched_setaffinity(0, x)

In [32]:
# chunks = []
# for i in range(1, len(d), 1000):
#    chunks.append(d[i:min(i + 1000, len(d))])

# res = Parallel(n_jobs=-1) (delayed(hash) (d) for d in chunks)

# def hash(d):
#     return [hashlib.sha256(str(x).encode('utf-8')).hexdigest()[:30] for x in d.itertuples(index=False, name=None)]

In [33]:
# Parallel(n_jobs=20)(delayed(sqrt)(i ** 2) for i in range(10000000))

In [34]:
# df = pd.DataFrame()
# df1 = pd.DataFrame()

In [35]:
# for i in range(1,55):
#     print(i)
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     df2 = pd.read_csv(fn)
#     df2.unit_sales[df2.unit_sales < 0] = 0
#     df3 = df2.set_index('date')
#     dates = np.unique(df2.date.values)
# #     for j, date in tqdm(enumerate(dates)):
#     df = df2.copy()
#     df1 = df3.copy()
#     df = df.iloc[::-1]
#     Parallel(n_jobs=10)(delayed(calculate_recent_stats)(date) for date in tqdm(dates))
# #         calculate_recent_stats(date, df, df1)
#     for days in [30, 90, 180, 365, 730]:
#         df = set_recent_df_stats(df ,df1, days)
#     df = df.iloc[::-1]
#     print(df.shape)
#     print(sum(df.isnull().any()==True)) # print NA
#     print(sum((df != 0).any(axis=0))) # print non-zero columns
#     fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
#     print(fn)
#     df.to_csv(fn, index=False)
#     del df2
#     del df3

In [36]:
# files_list = []
# for i in range(1,55):
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     files_list.append(fn)

In [37]:
# print('creating pool..')
# print(cpu_count())
# thread_pool = Pool(10)
# print('mapping pool..')
# thread_pool.map (calculate_recent_stats2, arg_list)

# print('joining pool..')

In [38]:
# from multiprocessing import Pool, cpu_count
def process_csv_file2(i):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn)
    df = pd.read_csv(fn)
    print(df.shape)
    df.unit_sales[df.unit_sales < 0] = 0 
    df = df.iloc[::-1]
    df1 = df.set_index('date')

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    for date in tqdm(trend_dates):
        calculate_recent_stats_not_perishable(date, df, df1)
    for days in [30, 90, 180, 365, 730]:
        df = set_recent_df_stats_nper(df, df1, days)
    df = df.iloc[::-1]
    
    df = df[(df.date.apply(np.datetime64) >= np.datetime64('2015-01-01')) & 
              (df.date.apply(np.datetime64) <= np.datetime64('2017-08-15'))]
    
    print(df.shape)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_nper.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    del df
    del df1

In [39]:
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [40]:
def process_csv_file1(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
    df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_nper_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_not_perishable(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_nper.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)

In [41]:
def process_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_nper_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_not_perishable(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_nper_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)

In [42]:
def process_test_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(i)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2017-08-16')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/test7_1_t_store{}_recent_stats_nper_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_not_perishable(date, df, fn_tmp)

    print(df.shape)
#     df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    df = df[(df.date.astype('str') >= str('2017-08-16')) & (df.date.astype('str') <= str('2017-08-31'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/test7_1_t_store{}_recent_stats_nper_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)

In [43]:
for i in range(1,55):
    process_test_csv_file(i) 

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 16/16 [01:39<00:00,  6.20s/it]


(2624569, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store1_recent_stats_nper_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store2.csv


100%|██████████| 16/16 [01:55<00:00,  7.23s/it]


(3050256, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store2_recent_stats_nper_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store3.csv


...

(62416, 28)
(16, 31)
0
38
../cache/test7_1_t_store52_recent_stats_nper_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store53.csv


100%|██████████| 16/16 [01:15<00:00,  4.71s/it]


(2000671, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store53_recent_stats_nper_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 16/16 [01:04<00:00,  4.01s/it]


(1711283, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store54_recent_stats_nper_1.csv
(62416, 57)


In [44]:
# Multi-threaded processes to utilize all available CPUs for this task. Note that many threads will block on IO
# so creating more than number of CPUs.    
# thread_pool = Pool(32)
# thread_pool.map (segment_pad_and_save_ct_scan_as_npz, dicom_folder_list)

# # Cleanup
# thread_pool.close()
# thread_pool.join_thread()

In [45]:
gc.collect()

1918

In [46]:
# i = 1
# fn = '../cache/train10_t_store{}.csv'.format(i)
# df = pd.read_csv(fn)
# df.unit_sales[df.unit_sales < 0] = 0
# df1 = df.set_index('date')
# dates = np.unique(df.date.values)
# for j, date in tqdm(enumerate(dates)):
#     if j > 0:
#         break
#     print(j)
#     calculate_recent_stats(date, df, df1)
# for days in [15, 30, 60, 90, 180, 365, 730, 1095, 1460]:
#     df = set_recent_df_stats(df ,df1, days)
# print(df.shape)
# print(sum(df.isnull().any()==True))
# print(sum((df != 0).any(axis=0)))
# fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
# print(fn)

In [47]:
# df.isnull().sum()

In [48]:
# for i in tqdm(range(1,55)):
#     fn_tmp = '../cache/train10_t_store{}_recent_stats_nper_tmp.csv'.format(i)
#     delete_file_if_exists(fn_tmp)
#     fn = '../cache/train10_t_store{}_recent_stats_nper.csv'.format(i)
#     df = pd.read_csv(fn)
#     df.to_csv(fn, float_format='%.5f', index=False)
#     del df